In [2]:
import problog
import pandas as pd

In [3]:
path = '/home/karel/Downloads/yago-wd-full-types(1).nt'

df = pd.read_csv(path, sep='\t',header=None,
                         names=None,
                         dtype=str,
                         usecols=[0, 1, 2])
df = df.drop_duplicates()

triples = df.values
# with open(path) as fin:
#         entity2id = dict()
#         id2entity = dict()
#         for line in fin:
#             print(line.strip().split('\t'))
#             h, r, t, _ = line.strip().split('\t')
#             print(h,r,t)
#             # entity2id[entity] = int(eid)
#             # id2entity[int(eid)] = entity


In [7]:
path = '/home/karel/Downloads/yago-wd-schema.nt'

df_schema = pd.read_csv(path, sep='\t',header=None,
                         names=None,
                         dtype=str,
                         usecols=[0, 1, 2])
df_schema = df_schema.drop_duplicates()

triples_schema = df_schema.values

In [8]:
triples

array([['<http://yago-knowledge.org/resource/Harald_Ringstorff>',
        '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
        '<http://yago-knowledge.org/resource/Human>'],
       ['<http://yago-knowledge.org/resource/Ecclesiastes>',
        '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
        '<http://yago-knowledge.org/resource/Religious_text>'],
       ['<http://yago-knowledge.org/resource/Ecclesiastes>',
        '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
        '<http://schema.org/Book>'],
       ...,
       ['<http://yago-knowledge.org/resource/Dingyuan_Township,_Qu_County>',
        '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
        '<http://yago-knowledge.org/resource/Human_settlement>'],
       ['<http://yago-knowledge.org/resource/Streatham_portrait>',
        '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
        '<http://schema.org/Painting>'],
       ['<http://yago-knowledge.org/resource/The_Girl_from_Nagasaki>',
        '<http://

In [9]:
triples_schema

array([['<http://schema.org/Chapter>',
        '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
        '<http://www.w3.org/2002/07/owl#Class>'],
       ['<http://schema.org/Chapter>',
        '<http://www.w3.org/2000/01/rdf-schema#label>', 'chapter@en'],
       ['<http://schema.org/Chapter>',
        '<http://www.w3.org/2000/01/rdf-schema#comment>',
        'One of the sections into which a book is divided. A chapter usually has a section number or a name.'],
       ...,
       ['_:schema-foundingDate-rdfs-range-owl-unionOf-xsd-date-xsd-dateTime-xsd-gYear-xsd-gYearMonth',
        '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
        '<http://www.w3.org/2000/01/rdf-schema#Datatype>'],
       ['_:schema-foundingDate-rdfs-range-owl-unionOf-xsd-date-xsd-dateTime-xsd-gYear-xsd-gYearMonth',
        '<http://www.w3.org/2002/07/owl#unionOf>',
        '_:list-xsd-date-xsd-dateTime-xsd-gYear-xsd-gYearMonth-1'],
       ['<http://schema.org/encodingFormat>',
        '<http://www.w3.org

In [4]:
from problog.program import PrologString
from problog.core import ProbLog
from problog import get_evaluatable
from problog.logic import Var, Term

# Functors (arguments will be added later)
subclass = Term('subclassOf')
subproperty = Term('subpropertyOf')
type = Term('typeOf')
range = Term('rangeOf')
domain = Term('domainOf')


# Literals
x = Term("x")
y = Term("y")

c = Term("c")
c1 = Term("c1")
c2 = Term("c2")
c3 = Term("c3")

r = Term("r")
r1 = Term("r1")
r2 = Term("r2")
r3 = Term("r3")

# Variables
triplet = Term("triplet")
X = Var('X')
Y = Var('Y')
Z = Var('Z')

In [6]:

################################################################################
## RDFS 2
################################################################################
type(X,c) << domain(r,c), r(X,Y) 

rdfs2 = PrologString(f"""
{domain}({r},{c}).
{r}({x},{y}).

{type}({X},{c}) :- {domain}({r},{c}), {r}({X},{Y}).

query({type}({x},{c})).
""")
get_evaluatable().create_from(rdfs2).evaluate()

{typeOf(x,c): 1.0}

In [188]:
################################################################################
## RDFS 3
################################################################################
type(Y,c) << range(r,c), r(X,Y) 

rdfs3 = PrologString(f"""
{range}({r},{c}).
{r}({x},{y}).

{type}({Y},{c}) :- {range}({r},{c}), {r}({X},{Y}).

query({type}({y},{c})).
""")

get_evaluatable().create_from(rdfs3).evaluate()

{typeOf(y,c): 1.0}

In [189]:
################################################################################
## RDFS 5
################################################################################
print("r1:", r1)
print("r2:", r2)
print("r3:", r3)

subproperty(r1,r3) << subproperty(r1,r2), subproperty(r2,r3)
rdfs5 = PrologString(f"""
{subproperty}({r1},{r2}).
{subproperty}({r2},{r3}).

{subproperty}({r1},{r3}) :- {subproperty}({r1},{r2}), {subproperty}({r2},{r3}).

query({subproperty}({r1},{r3})).
""")
get_evaluatable().create_from(rdfs5).evaluate()

r1: r1
r2: r2
r3: r3


{subpropertyOf(r1,r3): 1.0}

In [190]:
################################################################################
## RDFS 7
################################################################################
r2(X,Y) << subproperty(r1,r2), r1(X,Y) 

rdfs7 = PrologString(f"""
{r1}({x},{y}).
{subproperty}({r1},{r2}).

{r2}({X},{Y}) :- {subproperty}({r1},{r2}), {r1}({X},{Y}).

query({r2}({x},{y})).
""")

get_evaluatable().create_from(rdfs7).evaluate()

{r2(x,y): 1.0}

In [191]:
################################################################################
## RDFS 9
################################################################################
print("c1:", c1)
print("c2:", c2)
print("x:", x)
type(X,c2) << subclass(c1,c2), type(X,c1) 

rdfs9 = PrologString(f"""
{subclass}({c1},{c2}).
{type}({x},{c1}).

{type}({X},{c2}) :- {subclass}({c1},{c2}), {type}({X},{c1}).

query({type}({x},{c2})).
""")
get_evaluatable().create_from(rdfs9).evaluate()

c1: c1
c2: c2
x: x


{typeOf(x,c2): 1.0}

In [192]:
################################################################################
## RDFS 11
################################################################################
print("c1:", c1)
print("c2:", c2)
print("c3:", c3)

subclass(c1,c3) << subclass(c1,c2), subclass(c2,c3)
rdfs11 = PrologString(f"""
{subclass}({c1},{c2}).
{subclass}({c2},{c3}).

{subclass}({c1},{c3}) :- {subclass}({c1},{c2}), {subclass}({c2},{c3}).

query({subclass}({c1},{c3})).
""")

get_evaluatable().create_from(rdfs11).evaluate()

c1: c1
c2: c2
c3: c3


{subclassOf(c1,c3): 1.0}